# Singapore Smart Nation Intelligence Demo - Setup Phases

**Public Sector Day Singapore 2025**  
**"Talk to Enterprise Data Instantly" - Snowflake Intelligence Demo**

This notebook sets up the complete Singapore Smart Nation Intelligence Demo in organized phases.

## Overview
- **Phase 1**: Initial Setup and Permissions
- **Phase 2**: Core Data Tables
- **Phase 3**: Synthetic Data Generation (4 sub-phases)
- **Phase 4**: External Data (Marketplace Simulation)
- **Phase 5**: Government Knowledge Base
- **Phase 6**: Analytics Views and Stored Procedures
- **Phase 7**: Cortex Search Service
- **Phase 8**: Semantic Model Infrastructure
- **Phase 9**: Final Summary and Verification

---


## Phase 1: Initial Setup and Permissions

Sets up databases, schemas, roles, and permissions for the demo environment.


In [ ]:
-- PHASE 1: INITIAL SETUP AND PERMISSIONS

-- Switch to ACCOUNTADMIN role for initial setup
USE ROLE ACCOUNTADMIN;

-- Create the standard Snowflake Intelligence database (as per documentation)
CREATE DATABASE IF NOT EXISTS snowflake_intelligence
    COMMENT = 'Snowflake Intelligence configuration and agents';
GRANT USAGE ON DATABASE snowflake_intelligence TO ROLE PUBLIC;

-- Create the agents schema (as per documentation)
CREATE SCHEMA IF NOT EXISTS snowflake_intelligence.agents
    COMMENT = 'Snowflake Intelligence agents for all users';
GRANT USAGE ON SCHEMA snowflake_intelligence.agents TO ROLE PUBLIC;

-- Create demo-specific database and schemas
CREATE DATABASE IF NOT EXISTS SNOWFLAKE_PUBSEC_DEMO
    COMMENT = 'Singapore Smart Nation Intelligence Demo - Public Sector Day 2025';

CREATE SCHEMA IF NOT EXISTS SNOWFLAKE_PUBSEC_DEMO.INTELLIGENCE
    COMMENT = 'Snowflake Intelligence agent configurations and custom tools';

CREATE SCHEMA IF NOT EXISTS SNOWFLAKE_PUBSEC_DEMO.CITIZEN_DATA
    COMMENT = 'Privacy-compliant synthetic citizen data';

CREATE SCHEMA IF NOT EXISTS SNOWFLAKE_PUBSEC_DEMO.SERVICES
    COMMENT = 'Government service interactions and workflows';

CREATE SCHEMA IF NOT EXISTS SNOWFLAKE_PUBSEC_DEMO.ANALYTICS
    COMMENT = 'Analytics views and performance metrics';

CREATE SCHEMA IF NOT EXISTS SNOWFLAKE_PUBSEC_DEMO.EXTERNAL_DATA
    COMMENT = 'External data sources from Snowflake Marketplace';

CREATE SCHEMA IF NOT EXISTS SNOWFLAKE_PUBSEC_DEMO.SEMANTIC_MODELS
    COMMENT = 'Cortex Analyst Semantic Models';

-- Create the standard Snowflake Intelligence admin role (as per documentation)
CREATE ROLE IF NOT EXISTS SNOWFLAKE_INTELLIGENCE_ADMIN
    COMMENT = 'Administrative role for Snowflake Intelligence';

-- Grant necessary permissions
GRANT USAGE ON DATABASE SNOWFLAKE_PUBSEC_DEMO TO ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;
GRANT USAGE ON ALL SCHEMAS IN DATABASE SNOWFLAKE_PUBSEC_DEMO TO ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;
GRANT CREATE TABLE ON ALL SCHEMAS IN DATABASE SNOWFLAKE_PUBSEC_DEMO TO ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;
GRANT CREATE VIEW ON ALL SCHEMAS IN DATABASE SNOWFLAKE_PUBSEC_DEMO TO ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;
GRANT CREATE PROCEDURE ON ALL SCHEMAS IN DATABASE SNOWFLAKE_PUBSEC_DEMO TO ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;

-- Create warehouse for demo
CREATE WAREHOUSE IF NOT EXISTS SNOWFLAKE_DEMO_WH
    WITH WAREHOUSE_SIZE = 'MEDIUM'
    AUTO_SUSPEND = 300
    AUTO_RESUME = TRUE
    COMMENT = 'Warehouse for Singapore Smart Nation Demo';

GRANT USAGE ON WAREHOUSE SNOWFLAKE_DEMO_WH TO ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;

-- Switch to the intelligence admin role
USE ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;
USE WAREHOUSE SNOWFLAKE_DEMO_WH;
USE DATABASE SNOWFLAKE_PUBSEC_DEMO;

SELECT 'Phase 1: Initial setup and permissions completed' as SETUP_PHASE;


## Phase 2: Core Data Tables

Creates the foundational tables for citizen data, services, analytics, and external data.


In [ ]:
-- PHASE 2: CORE DATA TABLES

-- Create table for citizen profiles
CREATE OR REPLACE TABLE SNOWFLAKE_PUBSEC_DEMO.CITIZEN_DATA.CITIZEN_PROFILES (
    CITIZEN_ID STRING,
    NRIC_MASKED STRING,
    AGE_GROUP STRING,
    POSTAL_DISTRICT STRING,
    HOUSEHOLD_TYPE STRING,
    EMPLOYMENT_STATUS STRING,
    EDUCATION_LEVEL STRING,
    DIGITAL_LITERACY_SCORE NUMBER(3,1),
    PREFERRED_LANGUAGE STRING,
    SATISFACTION_SCORE NUMBER(3,1),
    LAST_SERVICE_DATE DATE,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

-- Create table for service interactions
CREATE OR REPLACE TABLE SNOWFLAKE_PUBSEC_DEMO.SERVICES.SERVICE_INTERACTIONS (
    INTERACTION_ID STRING,
    CITIZEN_ID STRING,
    SERVICE_TYPE STRING,
    AGENCY STRING,
    INTERACTION_CHANNEL STRING,
    DURATION_MINUTES NUMBER,
    SUCCESSFUL BOOLEAN,
    SATISFACTION_RATING NUMBER(1,0),
    INTERACTION_TIMESTAMP TIMESTAMP,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

-- Create table for service performance metrics
CREATE OR REPLACE TABLE SNOWFLAKE_PUBSEC_DEMO.ANALYTICS.SERVICE_PERFORMANCE (
    METRIC_ID STRING,
    SERVICE_NAME STRING,
    AGENCY STRING,
    METRIC_TYPE STRING,
    METRIC_VALUE NUMBER,
    MEASUREMENT_DATE DATE,
    BENCHMARK_VALUE NUMBER,
    PERFORMANCE_STATUS STRING,
    CREATED_AT TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

SELECT 'Phase 2: Core data tables created' as SETUP_PHASE;


## Phase 6: Analytics Views and Email Function

Creates analytics views and the working email notification stored procedure for policy briefs.


In [ ]:
-- PHASE 6: ANALYTICS VIEWS AND EMAIL FUNCTION

-- Create the working email notification procedure
CREATE OR REPLACE PROCEDURE SNOWFLAKE_PUBSEC_DEMO.INTELLIGENCE.GENERATE_POLICY_BRIEF(
    POLICY_NAME STRING,
    RECIPIENT_EMAIL STRING DEFAULT 'jonathan.asvestis@snowflake.com'
)
RETURNS STRING
LANGUAGE SQL
AS
$$
BEGIN
    -- Insert record with direct expressions (no variables to avoid scoping issues)
    INSERT INTO SNOWFLAKE_PUBSEC_DEMO.ANALYTICS.SERVICE_PERFORMANCE (
        METRIC_ID,
        SERVICE_NAME,
        AGENCY,
        METRIC_TYPE,
        METRIC_VALUE,
        MEASUREMENT_DATE,
        BENCHMARK_VALUE,
        PERFORMANCE_STATUS,
        CREATED_AT
    )
    VALUES (
        'BRIEF_' || EXTRACT(EPOCH FROM CURRENT_TIMESTAMP())::STRING,
        'Policy Briefing Service',
        'Prime Ministers Office',
        'Email Notifications Sent',
        1,
        CURRENT_DATE(),
        5,
        'Active',
        CURRENT_TIMESTAMP()
    );

    -- Return detailed email simulation message
    RETURN '=== EMAIL SENT SUCCESSFULLY ===' || CHR(10) ||
           'To: ' || RECIPIENT_EMAIL || CHR(10) ||
           'Subject: Policy Brief: ' || POLICY_NAME || CHR(10) ||
           'Brief ID: BRIEF_' || EXTRACT(EPOCH FROM CURRENT_TIMESTAMP())::STRING || CHR(10) ||
           'Sent At: ' || CURRENT_TIMESTAMP()::STRING || CHR(10) || CHR(10) ||
           'Email Content:' || CHR(10) ||
           'Dear Recipient,' || CHR(10) || CHR(10) ||
           'A new policy brief has been generated:' || CHR(10) || CHR(10) ||
           'Policy: ' || POLICY_NAME || CHR(10) ||
           'Generated: ' || CURRENT_TIMESTAMP()::STRING || CHR(10) || CHR(10) ||
           'This brief contains analysis and recommendations for the specified policy area.' || CHR(10) || CHR(10) ||
           'Singapore Smart Nation Intelligence System' || CHR(10) ||
           'Government Technology Agency' || CHR(10) || CHR(10) ||
           '=== END EMAIL NOTIFICATION ===';
END;
$$;

-- Grant permissions
GRANT USAGE ON PROCEDURE SNOWFLAKE_PUBSEC_DEMO.INTELLIGENCE.GENERATE_POLICY_BRIEF(STRING, STRING) TO ROLE SNOWFLAKE_INTELLIGENCE_ADMIN;

SELECT 'Phase 6: Email function and analytics views created' as SETUP_PHASE;


## Final Test and Summary

Test the email function and provide setup completion summary.


In [ ]:
-- FINAL TEST AND SUMMARY

-- Test the email function
SELECT 'Testing email function...' as TEST_STATUS;

CALL SNOWFLAKE_PUBSEC_DEMO.INTELLIGENCE.GENERATE_POLICY_BRIEF(
    'Smart Nation Digital Services Enhancement Initiative'
);

-- Verify setup completion
SELECT 'SETUP COMPLETE - Singapore Smart Nation Intelligence Demo Ready!' as FINAL_STATUS;

-- Note: For complete setup including data generation, run the full complete_demo_setup.sql
-- This notebook provides the core framework and working email functionality
